<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller # 9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

#### Librerías

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\user\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


#### Cargar datos

In [2]:
path = r'C:/Users/user/Documents/ESPECIALIZACION/LNP/Archivos/reviews_vidjew_es.csv'
data = pd.read_csv(path)
print(f"Tenemos {data.shape[0]} registros.")
data.sample(5)

Tenemos 1000 registros.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
977,es_0497934,product_es_0158405,reviewer_es_0305578,1,"nunca llego el producto, asi que no puedo deci...",nunca llego,es,video_games
882,es_0117244,product_es_0938917,reviewer_es_0477442,5,"Muy bonito, viene envasado en una cajita; punt...","Excelente producto, puntas redondeadas para qu...",es,jewelry
143,es_0178164,product_es_0765119,reviewer_es_0330451,1,"El Pendiente con la cadena viene roto, la base...",Lorena,es,jewelry
398,es_0693549,product_es_0471436,reviewer_es_0523134,4,"La cruz muy guapa, pero la cadena fina para el...",Javier,es,jewelry
638,es_0837248,product_es_0384682,reviewer_es_0956832,5,"Me han encantado, tienen un brillo azul precio...",Preciosos,es,jewelry


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['Pre-Procesado'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Procesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

#### Crear una represantación de los documentos en forma de diccionario

In [4]:
dictionary = Dictionary(data['Pre-Procesado'].values)

#### Filtrar palabras muy frecuentes o infrecuentes

In [5]:
dictionary.filter_extremes(no_below=5, no_above=0.5) 

#### Corpus

In [6]:
corpus = [dictionary.doc2bow(text) for text in data['Pre-Procesado'].values]

#### Entrenar el modelo

In [7]:
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics =5, passes=200)

In [8]:
model.print_topics(num_words=20)

[(0,
  '0.036*"producto" + 0.034*"día" + 0.029*"perfecto" + 0.025*"llegó" + 0.025*"llegado" + 0.023*"llego" + 0.020*"regalo" + 0.018*"buen" + 0.017*"sido" + 0.017*"pedido" + 0.016*"tiempo" + 0.016*"entrega" + 0.014*"si" + 0.013*"rápido" + 0.013*"envío" + 0.013*"nunca" + 0.012*"precio" + 0.012*"gracias" + 0.012*"mas" + 0.011*"artículo"'),
 (1,
  '0.082*"juego" + 0.020*"así" + 0.018*"pena" + 0.018*"jugar" + 0.018*"si" + 0.014*"perfecto" + 0.013*"español" + 0.012*"bastante" + 0.012*"mejor" + 0.012*"mal" + 0.012*"solo" + 0.011*"viene" + 0.011*"versión" + 0.010*"parte" + 0.010*"menos" + 0.010*"hijo" + 0.009*"cosa" + 0.008*"amazon" + 0.008*"xbox" + 0.008*"caja"'),
 (2,
  '0.032*"calidad" + 0.028*"pulsera" + 0.028*"queda" + 0.026*"bonito" + 0.025*"bien" + 0.022*"cadena" + 0.020*"bonita" + 0.020*"foto" + 0.020*"mas" + 0.017*"color" + 0.017*"pequeño" + 0.013*"precio" + 0.012*"parece" + 0.012*"colgante" + 0.012*"fina" + 0.011*"esperaba" + 0.011*"grande" + 0.011*"comprar" + 0.010*"todas" + 0.010*

###  `[25 pts]` Punto 3: Visualización de LDA

In [9]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

#### Resultados

In [34]:
d = dictionary.doc2bow(["juego", "calidad", "dia", "buena"])
topics = [(cluster[0]+1, cluster[1]) for cluster in model.get_document_topics(d)]
topics

[(1, 0.04026046),
 (2, 0.32110932),
 (3, 0.040278483),
 (4, 0.04011407),
 (5, 0.5582377)]

In [35]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

In [36]:
for t in range(0,5):
    top_name = f"topic_{t}"
    data[top_name] = data['Pre-Procesado'].apply(lambda doc: get_doc_top_n(doc, t))

In [37]:
data

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Procesado,topic_0,topic_1,topic_2,topic_3,topic_4
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]",0.033575,0.033334,0.033450,0.033478,0.866162
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw...",0.481963,0.033555,0.033392,0.417747,0.033335
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl...",0.028667,0.226875,0.028626,0.687013,0.028670
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble...",0.025157,0.025291,0.025214,0.898778,0.025570
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg...",0.245421,0.016746,0.016780,0.704036,0.016962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,es_0427672,product_es_0899883,reviewer_es_0474735,2,En la foto parece que la cadena sea más gruesa...,Cadena muy fina,es,jewelry,"[foto, parece, cadena, gruesa, fina, precio, p...",0.022323,0.022612,0.909786,0.022558,0.022746
996,es_0059540,product_es_0702564,reviewer_es_0754753,3,"Bien por su precio, la cadena demasiado fina.",E,es,jewelry,"[bien, precio, cadena, demasiado, fina]",0.033582,0.033447,0.864473,0.033823,0.034662
997,es_0303349,product_es_0678671,reviewer_es_0172313,5,Muy bien!! Ahora tengo organizados los pendien...,Bonito,es,jewelry,"[bien, ahora, organizados, pendientes]",0.050630,0.051603,0.051170,0.051915,0.794675
998,es_0659961,product_es_0294943,reviewer_es_0554554,4,"Las tapas para los joystics no están nada mal,...",No está mal pero sólo he probado las fundas.,es,video_games,"[tapas, joystics, mal, ayudan, montón, joystic...",0.015483,0.937872,0.015421,0.015687,0.015536


In [38]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91###  `[6 pts]` Punto 3: Entrenar modelo con el $K$ escogidom'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [33]:
for t in range(0,5):
    print(f"{color.PURPLE}{color.BOLD}----------------------------------------- TEMAS {t} -----------------------------------------{color.END}")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values(topic, ascending=False)['review_body'].values[:5]):
        print(f"{color.BLUE}{color.BOLD}{color.UNDERLINE}Artículo N. {i}{color.END}")
        print(f'{articulo[:500]}')
        print()
    print()

----------------------------------------- TEMAS 0 -----------------------------------------
Artículo N. 0
Buenas, quiero poner una queja del repartidor que ha traido este pedido, ha entregado el pedido pero a mi no me lo ha dado, ni he recibido ninguna llamada, simplemente ha dado un pedido de 64 euros a un completo desconocido, ruego expulsen de su empresa a ese impresentable, ya que es un pedido para uso personal y ha sido entregado a una direccion en la que ahora no hay viviendo nadie por equivocacion mia, se que he fallado pero lo mas normal es que me llamara o no entregara el pedido... Espero q

Artículo N. 1
El juego trae la caratula en italiano. Además, no trae el libro de instrucciones, y es algo que me ha extrañado y más siendo un juego nuevo. El envio muy lento. Pagué el envio urgente, de 5,99 € para tenerlo el dia 15, y me ha llegado hoy. Entiendo que esto no es culpa del vendedor, pero creo que no debería de utilizar la empresa ASM para los envios urgentes, ya que ofrecen u

###  `[3 pts]` Preguntas

***¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?****

Lo que puedo concluir es que cada modelo tiene sus ventajas y desventajas y se debe analizar cual utilizar según el dataset y los resultados que se buscan obtener con el modelo.

Ya que con los últimos tres talleres logramos analizar que a pesar de partir de la misma data los resultados de los 3 modelos arrojaron resultados diferentes, en el primero de clasificación de texto obtuvimos una clasificación de 2 etiquetas solamente, en el de K-means se amplió la salida a un número mayor de etiquetas haciendo una clasificación diferente a la primera y en esté último de LDA vemos que es más detallado y se agrupa por la frecuencia de los términos.

#### Elaborado por: Sandra Karina Osorio Ch. G51